In [1]:
from datasets.utils.logging import disable_progress_bar
from torch.utils.data import DataLoader, Subset, random_split
import torchvision
import torch
import torch.nn as nn
import torch.optim as optim

DATA_PATH='/tmp/data/cifar10'
DUMP_FILE_NAME = '/tmp/data/fed-data-NonIDD.pkl'

BATCH_SIZE = 64
NUM_CLIENTS = 10
NUM_CLASS = 10


transform = torchvision.transforms.Compose([
            torchvision.transforms.ToTensor(),
            torchvision.transforms.Normalize(
                (0.4914, 0.4822, 0.4465), 
                (0.2023, 0.1994, 0.2010)),
            ])

cifar10_train = torchvision.datasets.CIFAR10(
    root=DATA_PATH,
    train=True,
    transform=transform,
    download=True
)

cifar10_test = torchvision.datasets.CIFAR10(
    root=DATA_PATH,
    train=False,
    transform=transform,
    download=True
)

train_loader = DataLoader(cifar10_train, batch_size=64, shuffle=True)
test_loader = DataLoader(cifar10_test, batch_size=64, shuffle=False)

/usr/local/lib/python3.8/dist-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Files already downloaded and verified
Files already downloaded and verified


In [3]:
# Define VGG block
def vgg_block(num_convs, in_channels, out_channels):
    layers = []
    for _ in range(num_convs):
        layers += [nn.Conv2d(in_channels, out_channels, kernel_size=3, padding=1), nn.BatchNorm2d(out_channels), nn.ReLU(inplace=True)]
        in_channels = out_channels
    layers += [nn.MaxPool2d(kernel_size=2, stride=2)]
    return nn.Sequential(*layers)

# Define the VGG11bn model
class VGG11bn(nn.Module):
    def __init__(self, num_classes=10):
        super(VGG11bn, self).__init__()
        self.features = nn.Sequential(
            vgg_block(1, 3, 64),
            vgg_block(1, 64, 128),
            vgg_block(2, 128, 256),
            vgg_block(2, 256, 512),
            vgg_block(2, 512, 512),
        )
        self.classifier = nn.Sequential(
            nn.Linear(512, 4096),
            nn.ReLU(True),
            nn.Dropout(),
            nn.Linear(4096, 4096),
            nn.ReLU(True),
            nn.Dropout(),
            nn.Linear(4096, num_classes),
        )

    def forward(self, x):
        x = self.features(x)
        x = torch.flatten(x, 1)
        x = self.classifier(x)
        return x

In [4]:
# Training settings
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = VGG11bn(num_classes=10).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)


Epoch 1, Loss: 1.8966399873309123
Epoch 2, Loss: 1.4988476328380274
Epoch 3, Loss: 1.1953544190625096
Epoch 4, Loss: 0.9570493989283472
Epoch 5, Loss: 0.7900260783674772
Epoch 6, Loss: 0.6771278785698859
Epoch 7, Loss: 0.5832006204920961
Epoch 8, Loss: 0.49418837670475013
Epoch 9, Loss: 0.41230202577722347
Epoch 10, Loss: 0.3505897023584074


In [6]:

# Training loop
num_epochs = 10  # Adjust as needed
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
    print(f"Epoch {epoch+1}, Loss: {running_loss/len(train_loader)}")

Epoch 1, Loss: 0.2936335784880935
Epoch 2, Loss: 0.24922560699893842
Epoch 3, Loss: 0.21101587172121267
Epoch 4, Loss: 0.17824730002666678
Epoch 5, Loss: 0.14595967465463808
Epoch 6, Loss: 0.13911952501486824
Epoch 7, Loss: 0.1590074803692091
Epoch 8, Loss: 0.10324907992237611
Epoch 9, Loss: 0.09589396336215937
Epoch 10, Loss: 0.08508980711576675


In [5]:
# Testing for accuracy
model.eval()
correct = 0
total = 0
with torch.no_grad():
    for inputs, labels in test_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = model(inputs)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f'Accuracy of the network on the 10000 test images: {100 * correct / total} %')

Accuracy of the network on the 10000 test images: 80.88 %


In [1]:
from datasets.utils.logging import disable_progress_bar
from torch.utils.data import DataLoader, Subset, random_split
import torchvision
import numpy as np
import pickle
import torch

import test_socket_server.cifar10_SNN as cifar10_SNN

DATA_PATH='/tmp/data/cifar10'
NUM_CLIENTS = 8
NUM_CLASSES = 10
DUMP_FILE_NAME = '/tmp/data/fed-data-NonIDD.pkl'

transform = torchvision.transforms.Compose([
            torchvision.transforms.ToTensor(),
            torchvision.transforms.Normalize(
                (0.4914, 0.4822, 0.4465), 
                (0.2023, 0.1994, 0.2010)),
            ])

cifar10_train = torchvision.datasets.CIFAR10(
    root=DATA_PATH,
    train=True,
    transform=transform,
    download=True
)

cifar10_test = torchvision.datasets.CIFAR10(
    root=DATA_PATH,
    train=False,
    transform=transform,
    download=True
)

def prep_FL_NonIDD_data():
    # Calculate the size of each partition
    total_size = len(cifar10_train)
    indices = list(range(total_size))

    num_classes = 0
    for index in indices:
        num_classes = max(int(cifar10_train[index][1]) + 1, num_classes) 

    id_subset_of_class = [[] for i in range(num_classes)]

    for index in indices:
        category = int(cifar10_train[index][1])
        id_subset_of_class[category].append(index)
    
    id_subset_of_client = [[] for i in range(num_classes)]

    for i in range(NUM_CLIENTS):
        i0 = i
        i1 = (i + 1) % NUM_CLIENTS
        i2 = (i + 2) % NUM_CLIENTS
        i3 = (i + 3) % NUM_CLIENTS
        i4 = (i + 4) % NUM_CLIENTS
        s0, e0 = 0, int(len(id_subset_of_class[i0]) / 4)
        s1, e1 = int(len(id_subset_of_class[i1]) / 4), int(len(id_subset_of_class[i1]) / 4) * 2
        s2, e2 = int(len(id_subset_of_class[i2]) / 4) * 2, int(len(id_subset_of_class[i2]) / 4) * 3
        s3, e3 = int(len(id_subset_of_class[i3]) / 4) * 3, int(len(id_subset_of_class[i3]) / 4) * 4
        s4, e4 = int(len(id_subset_of_class[i4]) / 4) * 4, int(len(id_subset_of_class[i4]) / 4) * 5
        id_subset_of_client[i] = id_subset_of_class[i0][s0:e0] + \
                                id_subset_of_class[i1][s1:e1] + \
                                id_subset_of_class[i2][s2:e2] + \
                                id_subset_of_class[i3][s3:e3] + \
                                id_subset_of_class[i4][s4:e4]

    subsets = [Subset(cifar10_train, client_id)
                        for client_id in id_subset_of_client]

    # Create train/val for each partition and wrap it into DataLoader
    trainsets = []
    valsets = []
    for partition_id in range(NUM_CLIENTS):
        partition_train, partition_test = random_split(subsets[partition_id], [0.8, 0.2])
        
        trainsets.append(partition_train)
        valsets.append(partition_test)
    
    testset = cifar10_test
    return trainsets, valsets, testset
trainsets, valsets, testset = prep_FL_NonIDD_data()

/usr/local/lib/python3.8/dist-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Files already downloaded and verified
Files already downloaded and verified


In [2]:
for i in range(NUM_CLIENTS):
    print(len(trainsets[i]), [int(trainsets[i][j][1]) for j in range(len(trainsets[i]))])

4000 [3, 1, 1, 3, 1, 3, 3, 2, 2, 1, 1, 1, 3, 3, 0, 0, 0, 3, 3, 2, 0, 2, 1, 1, 2, 2, 0, 1, 3, 2, 3, 2, 0, 0, 0, 0, 2, 2, 3, 1, 1, 3, 1, 3, 2, 2, 1, 1, 0, 2, 2, 1, 2, 3, 0, 3, 1, 0, 1, 3, 3, 1, 3, 2, 0, 2, 2, 0, 1, 1, 1, 2, 3, 0, 0, 0, 0, 0, 1, 3, 1, 0, 2, 0, 0, 3, 2, 0, 2, 0, 0, 3, 2, 1, 2, 1, 3, 0, 2, 0, 1, 3, 1, 2, 0, 2, 2, 3, 2, 1, 1, 3, 0, 0, 1, 0, 0, 0, 1, 1, 2, 1, 1, 3, 2, 0, 3, 2, 0, 3, 0, 1, 1, 3, 1, 1, 2, 2, 1, 3, 3, 1, 0, 0, 0, 2, 3, 1, 3, 3, 1, 1, 1, 2, 1, 1, 1, 1, 1, 0, 0, 3, 1, 0, 0, 3, 1, 2, 0, 0, 1, 1, 3, 0, 3, 3, 2, 3, 2, 1, 3, 2, 2, 1, 2, 1, 2, 3, 1, 0, 1, 1, 2, 3, 2, 2, 0, 0, 2, 0, 3, 2, 3, 1, 2, 3, 0, 0, 3, 2, 0, 0, 2, 0, 1, 0, 1, 0, 1, 3, 0, 3, 2, 3, 2, 3, 1, 1, 0, 2, 0, 3, 0, 1, 1, 0, 0, 1, 1, 3, 0, 1, 0, 2, 1, 0, 3, 0, 2, 2, 2, 0, 2, 2, 3, 2, 3, 3, 1, 2, 0, 1, 3, 2, 2, 3, 2, 0, 1, 3, 2, 0, 2, 2, 0, 1, 3, 1, 1, 3, 1, 3, 1, 1, 1, 2, 2, 0, 3, 2, 1, 3, 0, 1, 3, 0, 0, 2, 1, 1, 2, 1, 0, 0, 2, 1, 0, 2, 2, 1, 3, 0, 0, 1, 3, 3, 0, 2, 2, 2, 0, 1, 3, 3, 1, 2, 3, 2, 0, 1, 0, 0

In [2]:
from dotenv import load_dotenv


In [4]:
load_dotenv('database/.env')

True

In [5]:
import os

In [8]:
float(os.getenv('NOISE_STD'))

0.01

In [9]:
os.getenv('DATA_PATH')

'/tmp/data/fed-data-IID.pkl'